# Import the Dependencies

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Data Collection & Pre-Processing

In [6]:
# loading the data from csv file to a pandas Dataframe
raw_mail_data = pd.read_csv('mail_data.csv')

In [9]:
print(raw_mail_data)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [11]:
# replace the null values with a null string
mail_data = raw_mail_data.where((pd.notnull(raw_mail_data)),'')

In [13]:
# printing the first 5 rows of the dataframe
mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [15]:
# checking the number of rows and columns in the dataframe
mail_data.shape

(5572, 2)

# Label Encoding

In [18]:
# label spam mail as 0;  ham mail as 1;

mail_data.loc[mail_data['Category'] == 'spam', 'Category',] = 0
mail_data.loc[mail_data['Category'] == 'ham', 'Category',] = 1

# Spam - 0
# Ham - 1

In [21]:
# Split features and labels
X = mail_data['Message']
Y = mail_data['Category'].astype(int)

In [23]:
print(X)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object


In [25]:
print(Y)

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: int32


# Splitting the data into training data & test data

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3)

In [30]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5572,)
(4457,)
(1115,)


# Feature Extraction

In [33]:
# transform the text data to feature vectors that can be used as input to the Logistic regression

feature_extraction = TfidfVectorizer(min_df = 1, stop_words='english', lowercase=True)

X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

# Convert to dense array
X_train_features = X_train_features.toarray()
X_test_features = X_test_features.toarray()

# Define fitness function for the Bat Algorithm

In [36]:
def fitness_function(weights, X, y):
    """Fitness function: Evaluate the classification performance."""
    # Apply weights to the dataset
    weighted_features = X * weights
    # Simple thresholding-based classifier
    predictions = np.where(np.sum(weighted_features, axis=1) > 0.5, 1, 0)
    # Calculate F1 score as the optimization metric
    return -f1_score(y, predictions)

# Implement the Bat Algorithm

In [39]:
def BAT(objf, lb, ub, dim, N, Max_iteration, X, y):
    # Initialization
    A = np.ones(N)   # Loudness
    r = np.ones(N)   # Pulse rate
    Qmin = 0         # Min frequency
    Qmax = 2         # Max frequency
    Q = np.zeros(N)  # Frequency
    v = np.zeros((N, dim))  # Velocities
    S = np.random.uniform(lb, ub, (N, dim))  # Solutions
    best = S[0]
    best_fitness = float('inf')
    
    for i in range(N):
        fitness = objf(S[i], X, y)
        if fitness < best_fitness:
            best_fitness = fitness
            best = S[i]
    
    Convergence_curve = []
    for t in range(Max_iteration):
        for i in range(N):
            Q[i] = Qmin + (Qmax - Qmin) * np.random.rand()
            v[i] += (S[i] - best) * Q[i]
            new_solution = S[i] + v[i]
            new_solution = np.clip(new_solution, lb, ub)
            
            if np.random.rand() > r[i]:
                new_solution = best + 0.1 * np.random.randn(dim)
            
            new_fitness = objf(new_solution, X, y)
            if new_fitness < best_fitness and np.random.rand() < A[i]:
                best = new_solution
                best_fitness = new_fitness
                A[i] *= 0.9  # Decrease loudness
                r[i] *= 1.1  # Increase pulse rate
        
        Convergence_curve.append(best_fitness)
    
    return best, -best_fitness, Convergence_curve

# Run the Bat Algorithm

In [42]:
dim = X_train_features.shape[1]  # Number of features
lb = np.zeros(dim)  # Lower bound of weights
ub = np.ones(dim)   # Upper bound of weights
N = 20              # Population size
Max_iteration = 50

best_weights, best_fitness, convergence = BAT(
    fitness_function, lb, ub, dim, N, Max_iteration, X_train_features, Y_train
)

# Apply the optimized weights to the test set

In [47]:
weighted_test_features = X_test_features * best_weights
test_predictions = np.where(np.sum(weighted_test_features, axis=1) > 0.5, 1, 0)

# Evaluation metrics

In [50]:
accuracy = accuracy_score(Y_test, test_predictions)
precision = precision_score(Y_test, test_predictions)
recall = recall_score(Y_test, test_predictions)
f1 = f1_score(Y_test, test_predictions)
conf_matrix = confusion_matrix(Y_test, test_predictions)

In [52]:
# Display results
print("Optimized Weights:", best_weights)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)

Optimized Weights: [0.41184512 0.81045893 0.84312507 ... 0.47099147 0.97190118 0.25839024]
Accuracy: 0.7883408071748879
Precision: 0.8507751937984496
Recall: 0.9145833333333333
F1 Score: 0.8815261044176707
Confusion Matrix:
 [[  1 154]
 [ 82 878]]
